<a href="https://colab.research.google.com/github/v4nema/bert_intent_recognition/blob/main/BERT_with_old_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
! pip install evaluate
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [21]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from datasets import load_dataset
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
import evaluate
import numpy as np
from transformers import pipeline
from evaluate import evaluator

In [3]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [4]:
accuracy = evaluate.load("accuracy")
model_name = "osiria/bert-base-italian-uncased"

In [80]:
ds_full = load_dataset('csv', delimiter="\t", data_files=['dataset/part1.tsv', 'dataset/part2.tsv'])
ds_train = ds_full['train'].class_encode_column("intent")
id2label = {i: v for i, v in enumerate(ds_train.info.features['intent'].names)}
ds_train[0]

{'text': 'come stai', 'intent': 12}

In [87]:
# ds = load_dataset('csv', delimiter="\t", data_dir='dataset')
ds = load_dataset('csv', delimiter="\t", data_files=['dataset/part1.tsv', 'dataset/part2.tsv'])
ds_train = ds['train'].rename_column('intent','label').class_encode_column("label")
id2label = {i: v for i, v in enumerate(ds_train.info.features['label'].names)}
# id2label = {i: v for i, v in enumerate(sorted(ds['train'].unique('intent')))}
# id2label = sorted(set(ds['train']['label']))
label2id = {v: i for i, v in id2label.items()}
num_labels = len(id2label)
# ds = ds.map(lambda ex: {'label': label2id[ex['intent']]})

ds = ds_train.train_test_split(seed=42,
                               test_size=0.2,
                               stratify_by_column='label')

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenized_ds = ds.map(lambda ex: tokenizer(ex["text"], truncation=True), batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
)
# model.layers[0].trainable = False

batch_size = 16
num_epochs = 50
batches_per_epoch = len(ds["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

tf_train_set = model.prepare_tf_dataset(
    tokenized_ds["train"],
    shuffle=True,
    batch_size=batch_size,
    tokenizer=tokenizer,
    # collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_ds["test"],
    shuffle=False,
    batch_size=batch_size,
    tokenizer=tokenizer,
    # collate_fn=data_collator,
)

model.compile(optimizer=optimizer)

Casting to class labels:   0%|          | 0/3098 [00:00<?, ? examples/s]

Map:   0%|          | 0/2478 [00:00<?, ? examples/s]

Map:   0%|          | 0/620 [00:00<?, ? examples/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using

In [88]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='ckpt/best_model',
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

In [89]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=[metric_callback, checkpoint_callback])

Epoch 1/50
154/154 [==============================] - 92s 280ms/step - loss: 4.2149 - val_loss: 3.4725 - accuracy: 0.3887
Epoch 2/50
154/154 [==============================] - 25s 159ms/step - loss: 3.0100 - val_loss: 2.3045 - accuracy: 0.6758
Epoch 3/50
154/154 [==============================] - 25s 161ms/step - loss: 2.0470 - val_loss: 1.5982 - accuracy: 0.7839
Epoch 4/50
154/154 [==============================] - 25s 161ms/step - loss: 1.4237 - val_loss: 1.1808 - accuracy: 0.8452
Epoch 5/50
154/154 [==============================] - 25s 161ms/step - loss: 1.0237 - val_loss: 0.8963 - accuracy: 0.8887
Epoch 6/50
154/154 [==============================] - 25s 162ms/step - loss: 0.7445 - val_loss: 0.7307 - accuracy: 0.8968
Epoch 7/50
154/154 [==============================] - 26s 172ms/step - loss: 0.5672 - val_loss: 0.5935 - accuracy: 0.9048
Epoch 8/50
154/154 [==============================] - 30s 192ms/step - loss: 0.4255 - val_loss: 0.5073 - accuracy: 0.9161
Epoch 9/50
154/154 [====

# Save & load last model with HuggingFace api

In [90]:
model.save_pretrained('intent_bert', from_pt=True)

In [91]:
!zip -r intent_bert.zip intent_bert

  adding: intent_bert/ (stored 0%)
  adding: intent_bert/config.json (deflated 71%)
  adding: intent_bert/tf_model.h5 (deflated 7%)


In [92]:
last_model = TFAutoModelForSequenceClassification.from_pretrained('intent_bert')

Some layers from the model checkpoint at intent_bert were not used when initializing TFBertForSequenceClassification: ['dropout_151']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at intent_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [93]:
# @title Test last model
text = "mi spieghi i titoli azionari" #@param {type:"string"}
inputs = tokenizer(text, return_tensors="tf")
logits = model(**inputs).logits
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
predicted_class_id, model.config.id2label[predicted_class_id]

(31, '37_share_education_share')

In [94]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [95]:
pipe(text)

[{'label': '37_share_education_share', 'score': 0.9900301694869995}]

In [96]:
task_evaluator = evaluator("text-classification")
task_evaluator.compute(
    model_or_pipeline=pipe,
    data=ds["test"],
    label_mapping=label2id
)

{'accuracy': 0.9274193548387096,
 'total_time_in_seconds': 232.34090225899945,
 'samples_per_second': 2.668492693158529,
 'latency_in_seconds': 0.3747433907403217}

In [98]:
model2.load_weights('ckpt/best_model')
pipe2 = pipeline("text-classification", model=model2, tokenizer=tokenizer)

In [99]:
task_evaluator = evaluator("text-classification")
task_evaluator.compute(
    model_or_pipeline=pipe2,
    data=ds["test"],
    label_mapping=label2id
)

{'accuracy': 0.932258064516129,
 'total_time_in_seconds': 238.23896124400017,
 'samples_per_second': 2.602429076934259,
 'latency_in_seconds': 0.38425638910322607}

In [58]:
for i in model.config.id2label:
  print(i, model.config.id2label[i], id2label[i])

0 100_wt_what_time 100_wt_what_time
1 101_yes 101_yes
2 10_block_education_blockchain 10_block_education_blockchain
3 11_bond_education_bond 11_bond_education_bond
4 12_cbdc_education_cbdc 12_cbdc_education_cbdc
5 13_cif_education_cif 13_cif_education_cif
6 14_commo_education_commodities 14_commo_education_commodities
7 15_crypto_education_crypto 15_crypto_education_crypto
8 16_cycli_education_cyclical_sectors 16_cycli_education_cyclical_sectors
9 17_deri_education_derivate 17_deri_education_derivate
10 18_diversification_education_diversification 18_diversification_education_diversification
11 19_etf_education_etf 19_etf_education_etf
12 1_how_about_me_howareyou 1_how_about_me_howareyou
13 20_market_education_financial_market 20_market_education_financial_market
14 21_title_education_finantial_title 21_title_education_finantial_title
15 22_horizon_education_horizon 22_horizon_education_horizon
16 23_inflation_education_inflation 23_inflation_education_inflation
17 24_interest_educatio

In [70]:
ds['train'].map(lambda ex: {'label2id': label2id[ex['intent']]})[8]
type(ds['train'].info.features['label'])
ds['train'].info.features['label'].names

['0',
 '1',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99']

In [71]:
test = ds.map(lambda e: {'test': e['intent']}).class_encode_column("test")

Map:   0%|          | 0/2478 [00:00<?, ? examples/s]

Map:   0%|          | 0/620 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2478 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/620 [00:00<?, ? examples/s]

In [76]:
test['train'][10]

{'text': 'mi spieghi i titoli azionari',
 'intent': '37_share_education_share',
 'label': 26,
 'test': 31}